In [1]:
%pip install ipywidgets
# %pip install solara

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 27.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 38.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.1/241.1 kB 5.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 1.0 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 4.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output

# Artist selection widget
artist_toggle_options = ['At most k songs', 'At least k songs', 'None of', '--']
artist_toggle = widgets.Dropdown(
    options=artist_toggle_options,
    description='Preferences:',
    disabled=False,
    value='--'
)

# Number of songs for artist
artist_song_count = widgets.IntSlider(
    value=1,
    min=0,
    max=100,  # Adjust maximum as needed
    step=1,
    description='Songs Count:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='25%'),
    style={'description_width': 'initial'}
)

artist_name = widgets.Text(description='Artist name:')
song_name = widgets.Text(description='Song name:')

# artist_and_count = widgets.HBox([artist_toggle])
# artist_and_count.layout.align_items = 'center'

# intialize the output - second widget
out = Output()

def changed(change):
    '''
    Monitor change in the first widget
    '''
    global out

    if artist_toggle.value == 'At least k songs':
        out.append_display_data(artist_song_count)
#         display(out)
    elif artist_toggle.value == 'At most k songs':
        out.append_display_data(artist_song_count)
#         display(out)
    elif artist_toggle.value == 'None of':
        out.append_display_data(artist_name)
#         display(out)
    else:
        out.clear_output()
        out = Output()
    
    display(out)
    out.clear_output()   

                
# monitor the friends widget for changes
artist_toggle.observe(changed, 'value')

playlist_length_toggle = widgets.ToggleButtons(
    description='Playlist length:',
    options= ['Minutes', 'Number of Songs'])

# # Playlist length in minutes
# playlist_length_mins = widgets.IntRangeSlider(
#     value=[0, 120],
#     min=0,
#     max=120,  # 2 hours in minutes
#     step=10,  # step in minutes
#     description='Playlist Length (min):',
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
#     style={'description_width': 'initial'},
# )

# # Playlist length in number of songs
# playlist_length_num_songs = widgets.IntSlider(
#     value=10,
#     min=2,
#     max=100,
#     step=1,
#     description='Playlist Length (number of songs):',
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
#     style={'description_width': 'initial'},
# )

# # intialize the output - second widget
# out = Output()

# def changed(change):
#     '''
#     Monitor change in the first widget
#     '''
#     global out
#     if playlist_length_toggle.value == 'Minutes':  
#         out.append_display_data(playlist_length_mins)
#         # display(out)
#     elif playlist_length_toggle.value == 'Number of Songs':
#         out.append_display_data(playlist_length_num_songs)
#         # display(out)
#     else:
#         out.clear_output()
#         out = Output()
    
#     display(out)
#     out.clear_output()    
                
# monitor the friends widget for changes
playlist_length_toggle.observe(changed, 'value')


# BPM range widget
bpm_range = widgets.IntRangeSlider(
    value=[60, 180],  # Default BPM range
    min=0,
    max=300,
    step=1,
    description='BPM Range:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Loudness widget
loudness_range = widgets.FloatRangeSlider(
    value=[-60.0, 0.0],
    min=-60.0,
    max=0.0,
    step=0.1,
    description='Loudness:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# Liveness widget
liveness_range = widgets.FloatRangeSlider(
    value=[0.0, 1.0],
    min=0.0,
    max=1.0,
    step=0.01,
    description='Liveness:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

# Mode widget
mode_options = ['Major', 'Minor', 'Both']
mode_selection = widgets.Dropdown(
    options=mode_options,
    description='Mode:',
    disabled=False,
)

# Display widgets
display(artist_toggle, bpm_range, loudness_range, liveness_range, mode_selection)

Dropdown(description='Preferences:', index=3, options=('At most k songs', 'At least k songs', 'None of', '--')…

IntRangeSlider(value=(60, 180), continuous_update=False, description='BPM Range:', max=300)

FloatRangeSlider(value=(-60.0, 0.0), continuous_update=False, description='Loudness:', max=0.0, min=-60.0, rea…

FloatRangeSlider(value=(0.0, 1.0), continuous_update=False, description='Liveness:', max=1.0, step=0.01)

Dropdown(description='Mode:', options=('Major', 'Minor', 'Both'), value='Major')

IntSlider(value=1, continuous_update=False, description='Songs Count:', layout=Layout(width='25%'), style=Slid…

Output(outputs=({'output_type': 'display_data', 'data': {}, 'metadata': {}},))

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/spotify_playlists.tsv', sep='\t', index_col=0)
df = df.sample(100)
# assign a random similarity score
df["similarity"] = np.random.rand(len(df))
df

,artist_name,track_name,uri,num_plays,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,similarity
8077,Kanye West,Homecoming,spotify:track:4iz9lGMjU1lXS51oPmUmTe,2,0.667,0.747,1.0,-7.059,1.0,0.1890,...,0.1150,0.9180,86.917,audio_features,4iz9lGMjU1lXS51oPmUmTe,https://api.spotify.com/v1/tracks/4iz9lGMjU1lX...,https://api.spotify.com/v1/audio-analysis/4iz9...,203493.0,4.0,0.498390
9690,Shakira,About Cut Me Deep - Commentary,spotify:track:1jqjwhu79wSGsX6Cxd6r5G,1,0.606,0.156,2.0,-26.633,1.0,0.8740,...,0.2340,0.3430,80.903,audio_features,1jqjwhu79wSGsX6Cxd6r5G,https://api.spotify.com/v1/tracks/1jqjwhu79wSG...,https://api.spotify.com/v1/audio-analysis/1jqj...,22640.0,4.0,0.132399
15493,Outwild,With Clouds Below,spotify:track:7hwSaTdDujbwXGGp72hssA,1,0.510,0.836,9.0,-4.975,1.0,0.0273,...,0.0878,0.0626,109.936,audio_features,7hwSaTdDujbwXGGp72hssA,https://api.spotify.com/v1/tracks/7hwSaTdDujbw...,https://api.spotify.com/v1/audio-analysis/7hwS...,209455.0,4.0,0.548028
7505,Shing02,Luv (Sic),spotify:track:6TxDFfXqraQlaCS2DNfy5w,2,0.917,0.721,1.0,-4.216,1.0,0.1470,...,0.6630,0.8930,93.044,audio_features,6TxDFfXqraQlaCS2DNfy5w,https://api.spotify.com/v1/tracks/6TxDFfXqraQl...,https://api.spotify.com/v1/audio-analysis/6TxD...,288760.0,4.0,0.196581
461,Jeremih,Down On Me,spotify:track:0x3TibRgnSqzkcwrqBd4iX,97,0.700,0.598,2.0,-7.783,1.0,0.1140,...,0.1110,0.5940,160.041,audio_features,0x3TibRgnSqzkcwrqBd4iX,https://api.spotify.com/v1/tracks/0x3TibRgnSqz...,https://api.spotify.com/v1/audio-analysis/0x3T...,228453.0,4.0,0.908027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3821,Chris Tomlin,"O, Holy Night",spotify:track:175ywr52ZDXpQGbyQhzfcr,8,0.275,0.305,0.0,-8.699,1.0,0.0309,...,0.0907,0.0733,149.311,audio_features,175ywr52ZDXpQGbyQhzfcr,https://api.spotify.com/v1/tracks/175ywr52ZDXp...,https://api.spotify.com/v1/audio-analysis/175y...,376267.0,3.0,0.611390
15334,Mooski,Track Star,spotify:track:5hChuUMe5iHfjkXgzj1D4b,1,0.641,0.598,7.0,-5.700,0.0,0.3240,...,0.0921,0.2850,131.459,audio_features,5hChuUMe5iHfjkXgzj1D4b,https://api.spotify.com/v1/tracks/5hChuUMe5iHf...,https://api.spotify.com/v1/audio-analysis/5hCh...,199706.0,5.0,0.018044
4258,The Millennial Club,complicated/overrated,spotify:track:44UGWn9xCTjeHJ2LO52uNT,6,0.727,0.686,4.0,-5.844,1.0,0.1540,...,0.0619,0.6230,95.097,audio_features,44UGWn9xCTjeHJ2LO52uNT,https://api.spotify.com/v1/tracks/44UGWn9xCTje...,https://api.spotify.com/v1/audio-analysis/44UG...,148143.0,4.0,0.244524
5511,Brent Faiyaz,First World Problemz / Nobody Carez,spotify:track:042wd4CrF4WTAwiKs0F7Ve,3,0.582,0.502,3.0,-11.053,0.0,0.1020,...,0.1600,0.5550,85.629,audio_features,042wd4CrF4WTAwiKs0F7Ve,https://api.spotify.com/v1/tracks/042wd4CrF4WT...,https://api.spotify.com/v1/audio-analysis/042w...,350549.0,4.0,0.288229


In [4]:
df.columns

Index(['artist_name', 'track_name', 'uri', 'num_plays', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'similarity'],
      dtype='object')

In [5]:
artist_and_count.children[0].value

NameError: name 'artist_and_count' is not defined

In [ ]:
# constraints 
# 1) order by similarities 
# 2) apply contraints from widgets 
df = df.sort_values("similarity", ascending=False)
df = df[(df["tempo"] > bpm_range.value[0]) & (df["tempo"] < bpm_range.value[1])]
df = df[(df["loudness"] > loudness_range.value[0]) & (df["loudness"] < loudness_range.value[1])]
df = df[df["mode"] == mode_selection.value]
# todo: do all the range/equality contraints, 
# then do the "hard" ones like artist_song_count and playlist_length and num_song in that order probably 
# make sure artist_count <= num_songs
df

In [6]:
df["tempo"].describe()

count    100.000000
mean     113.804330
std       28.840826
min       57.750000
25%       88.036250
50%      111.140500
75%      132.510500
max      191.825000
Name: tempo, dtype: float64

In [7]:
filtering_criterion = {
    "Author": "SZA",
    "ms_played": lambda x: x < 100_000
}

In [8]:
import pandas as pd

# Assuming you have a dataframe named 'songs_df' with columns 'song_name', 'artist', 'duration_minutes'
# Example dataframe creation
data = {'song_name': ['Song1', 'Song2', 'Song3', 'Song4', 'Song5'],
        'artist': ['Bob', 'Alice', 'Bob', 'Charlie', 'Alice'],
        'duration_minutes': [3, 4, 2, 5, 3]}
songs_df = pd.DataFrame(data)

def select_songs(songs_df, playlist_length=None, artist_constraint=None, artist_name=None, artist_condition=None):
    # Constraint 3: Include the specified number of songs for the given artist and condition
    if artist_constraint and artist_name and artist_condition is not None:
        artist_songs = songs_df[songs_df['artist'] == artist_name]
        if artist_condition == 'at least':
            artist_songs = artist_songs.head(max(artist_constraint, len(artist_songs)))
        elif artist_condition == 'at most':
            artist_songs = artist_songs.head(min(artist_constraint, len(artist_songs)))
        elif artist_condition == 'none':
            artist_songs = pd.DataFrame(columns=songs_df.columns)
        else:
            print("Invalid artist condition.")
            return None
    else:
        artist_songs = pd.DataFrame(columns=songs_df.columns)

    # Constraint: Select songs until the playlist reaches the specified length +/- 10 minutes
    remaining_duration = max(0, playlist_length - artist_songs['duration_minutes'].sum())
    selected_songs = songs_df.head(len(artist_songs))
    while remaining_duration > 0:
        next_song = songs_df.iloc[len(selected_songs)]
        if abs(remaining_duration - next_song['duration_minutes']) <= 10:
            selected_songs = pd.concat([selected_songs, next_song.to_frame().transpose()])
            break
        elif remaining_duration - next_song['duration_minutes'] > 0:
            selected_songs = pd.concat([selected_songs, next_song.to_frame().transpose()])
            remaining_duration -= next_song['duration_minutes']
        else:
            break

    # Combine selected songs based on the order of constraints
    final_selection = pd.concat([artist_songs, selected_songs])

    return final_selection

# Example usage:
selected_subset = select_songs(songs_df, playlist_length=17, artist_constraint=2, artist_name='Bob', artist_condition='at least')
print(selected_subset)


  song_name artist duration_minutes
0     Song1    Bob                3
2     Song3    Bob                2
0     Song1    Bob                3
1     Song2  Alice                4
2     Song3    Bob                2


In [9]:
%pip install python-constraint

Note: you may need to restart the kernel to use updated packages.


In [10]:
# FROM JESS'S Inference.ipynb

# function to get top-n most similar tracks
def get_most_similar_tracks(track_name, n=10, tokenizer=tokenizer, embedding_weights=embedding_weights):
    
    # get track embedding
    track_idx = tokenizer.word_index[track_name]
    track_vector = embedding_weights[track_idx, :].reshape(1, -1)

    # compute similarities against other tracks
    similarities = np.dot(track_vector, embedding_weights.T) / (np.linalg.norm(track_vector) * np.linalg.norm(embedding_weights, axis=1))
    similarities = similarities.reshape(-1)

    # get most similar tracks' indices
    most_similar_idxs = np.argpartition(similarities, -(n+1))[-(n+1):]
    most_similar_idxs = most_similar_idxs[np.argsort(similarities[most_similar_idxs])][::-1][1:]

    # print most similar tracks, along with their positions in training data
    print("top {} tracks most similar to '{}' (pos. {}):".format(n, track_name, track_idx))
    for idx in most_similar_idxs:
        print("- (sim. {:.3f}): '{}' (pos. {})".format(similarities[idx], tokenizer.index_word[idx], idx))

NameError: name 'tokenizer' is not defined